# Real-Time Object Detection using YOLO

### Import Necessary Libraries

In [ ]:
# Importing required libraries
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
import time

### Load the YOLO Model

In [ ]:
# Load the trained YOLO model (ensure your 'best.pt' file is in the same directory)
model = YOLO("runs/detect/train/weights/best.pt")

# Waste classes (these should match the labels your model was trained on)
classes = ['biological', 'cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

### Set Up Webcam Capture

In [ ]:
# Set up webcam capture
cap = cv2.VideoCapture(0)  # 0 is the default camera

# Check if webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

### Real-Time Inference and Display in Jupyter Lab

In [ ]:
# Define the function to detect objects and take a snapshot every 10 seconds
def detect_objects():
    start_time = time.time()  # Store the start time
    
    try:
        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame.")
                break

            current_time = time.time()

            # If 10 seconds have passed, process and show the snapshot
            if current_time - start_time >= 10:
                # Perform inference
                results = model.predict(source=frame, verbose=False)
                output = results[0]

                # Draw bounding boxes and labels
                for box, conf, cls in zip(output.boxes.xyxy, output.boxes.conf, output.boxes.cls):
                    x1, y1, x2, y2 = map(int, box)
                    class_name = classes[int(cls)]
                    color = (0, 255, 0)  # Green boxes for all classes

                    # Draw bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                    # Label with class name and confidence
                    label = f"{class_name}: {conf:.2f}"
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

                # Convert BGR to RGB for display in Jupyter (OpenCV uses BGR by default)
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Display the resulting frame using matplotlib
                plt.imshow(frame_rgb)
                plt.axis('off')  # Hide axes
                plt.show()

                start_time = current_time  # Reset the start time for next snapshot

            # Exit if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        # Release the webcam and close any OpenCV windows
        cap.release()
        cv2.destroyAllWindows()

# Run the detection function (snapshot every 10 seconds)
detect_objects()

### Stopping the Webcam Capture

To stop the webcam, you can press `q` on your keyboard. It will gracefully close the webcam feed and all windows.